## 필요 라이브러리 설치

!pip install BeautifulSoup4

!pip install pandas

!pip install numpy

## 기본 html 파싱

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
res = requests.get('https://musclewiki.com/Directory')
soup = BeautifulSoup(res.content, 'html.parser')

lst = soup.find_all('tr')

## 필요 데이터 추출

In [56]:
muscle = []
exercise = []
equipment = []
difficulty = []
link = []

for item in lst:
    txt = [tmp for tmp in item.get_text().split('\n') if tmp]
    if 'Exercises' in item.get_text():
        muscle.append(txt[0].strip()[:-10])
        exercise.append([])
        equipment.append([])
        difficulty.append([])
        link.append([])
        
    else:
        exercise[-1].append(txt[0])
        equipment[-1].append(txt[-2])
        difficulty[-1].append(txt[-1])
        tmp = 'https://musclewiki.com' + item.find('a')['href']
        link[-1].append(tmp)

## 데이터프레임 생성

In [77]:
df_lst = []

for i in range(len(muscle)):
    tmp_df = pd.DataFrame([exercise[i], equipment[i], difficulty[i], link[i]], 
                          index = ['exercise', 'equipment', 'difficulty', 'link']).transpose()
    df_lst.append(tmp_df)

## https://www.musclewiki.com + 이미지주소 하면 움짤 주소임!

df = df_lst[0]
a = df.loc[0, 'link']
b = requests.get(a)
s = BeautifulSoup(b.content, 'html.parser')
s.find(class_ = 'exercise-images-grid').find('img')['src']

a = df_lst[4]
r = requests.get(a.loc[0, 'link'])
s = BeautifulSoup(r.content, 'html.parser')
s.get_text()

for df in df_lst:
    for i in range(len(df)):
        tmp_link = df.loc[i, 'link']
        tmp_res = requests.get(tmp_link)
        tmp_soup = BeautifulSoup(tmp_res.content, 'html.parser')
        
        
        if '404' in tmp_soup.get_text():
            df.loc[i, 'img_link'] = '이미지 주소 오류'
        else:

            images = tmp_soup.find(class_ = 'exercise-images-grid').find_all('img')
            print(i, len(images))
    print()

In [78]:
for df in df_lst:
    for i in range(len(df)):
        link = df.loc[i, 'link']
        tmp_res = requests.get(link)
        tmp_soup = BeautifulSoup(tmp_res.content, 'html.parser')
        
        if '404' in tmp_soup.get_text():
            df.loc[i, 'img_link'] = '이미지 주소 오류'
        else:
            img_link = tmp_soup.find(class_ = 'exercise-images-grid').find('img')
            df.loc[i, 'img_link'] = 'https://www.musclewiki.com' + img_link['src']

In [93]:
df_lst[14]

,exercise,equipment,difficulty,link,img_link
0,Barbell Landmine Side Bend,Barbell,Intermediate,https://musclewiki.com/Barbell/Male/Abdominals...,https://www.musclewiki.com/media/uploads/barbe...
1,Dumbbell Woodchopper,Dumbbells,Advanced,https://musclewiki.com/Dumbbells/Male/Abdomina...,https://www.musclewiki.com/media/uploads/male-...
2,Kettlebell Woodchopper,Kettlebells,Beginner,https://musclewiki.com/Kettlebells/Male/Abdomi...,https://www.musclewiki.com/media/uploads/kettl...
3,Cable Wood Chopper,Cables,Intermediate,https://musclewiki.com/Cables/Male/Abdominals/...,https://www.musclewiki.com/media/uploads/cable...
4,Wood Chopper,Band,Advanced,https://musclewiki.com/Band/Male/Abdominals/Wo...,https://www.musclewiki.com/media/uploads/band-...
5,Russian Twist,Dumbbells,Intermediate,https://musclewiki.com/Dumbbells/Male/Abdomina...,https://www.musclewiki.com/media/uploads/male-...
6,Kettlebell Russian Twist,Kettlebells,Beginner,https://musclewiki.com/Kettlebells/Male/Abdomi...,https://www.musclewiki.com/media/uploads/kettl...
7,Pallof Press,Band,Beginner,https://musclewiki.com/Band/Male/Abdominals/Pa...,https://www.musclewiki.com/media/uploads/band-...
8,Windmill,Kettlebells,Beginner,https://musclewiki.com/Kettlebells/Male/Abdomi...,https://www.musclewiki.com/media/uploads/kettl...
9,Cable Pallof Press,Cables,Beginner,https://musclewiki.com/Cables/Male/Abdominals/...,https://www.musclewiki.com/media/uploads/cable...


df_lst[0].iloc[0]['link']
df_lst[0].loc[0, 'img_front'] = 'hello'
df_lst[0].loc[0, 'img_side'] = 'world'
df_lst[0]

## 난이도로 구분
- 추후 그룹화를 통해 기구 구분하여 각각 저장

In [30]:
beginner_lst = []
senior_lst = []

